In [1]:
import pandas as pd

In [2]:
data = pd.read_json("2018-01.json");

prend la colonne games

In [4]:
games = data["games"]

on recupere l'historique des cartes sorties pour chaque partie
On filtre si le champs card_history existe ou si il est vide

In [14]:
card_history = []
for i in range(0,len(games)):
    g = games[i]
    if not(g.has_key("card_history")) or g["card_history"] != []:
      card_history.append(pd.DataFrame(g["card_history"]))

obtenir toutes cartes me pour la premier partie

c = card_history[0]
me = c[c.player=="me"]
op = c[c.player=="opponent"]
me = me['card']
op = op['card']

sentances = []
sentances.append(" ".join([i["name"].replace(" ","_") for i in me]))
sentances.append(" ".join([i["name"].replace(" ","_") for i in op]))

print sentances

pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent"
puis on recupère le nom des cartes pour crée des phrases pour les passer ensuite au word2vec plus tard.
Pour les noms composés on remplace les espace par des inderscore pour les considérer commme un mot entier  

In [15]:
sentances = []

In [17]:
for i in range(0,len(card_history)):
    c = card_history[i]
    me = c[c.player=="me"]
    op = c[c.player=="opponent"]
    me = me["card"]
    op = op['card']
    sentances.append(" ".join([i["name"].replace(" ","_") for i in me]))
    sentances.append(" ".join([i["name"].replace(" ","_") for i in op]))

probleme 

In [48]:
card_history[361]

,card,player,turn
0,"{u'mana': 1, u'id': u'UNG_856', u'name': u'Hal...",me,1
1,"{u'mana': 2, u'id': u'LOOT_413', u'name': u'Pl...",opponent,2
2,"{u'mana': 2, u'id': u'LOOT_033', u'name': u'Ca...",me,2
3,"{u'mana': 2, u'id': u'CS2_034', u'name': u'Fir...",opponent,3
4,"{u'mana': 3, u'id': u'CFM_781', u'name': u'Sha...",me,3
5,"{u'mana': 2, u'id': u'CS2_034', u'name': u'Fir...",opponent,4
6,"{u'mana': 2, u'id': u'NEW1_021', u'name': u'Do...",opponent,4
7,"{u'mana': 5, u'id': u'LOOT_130', u'name': u'Ar...",opponent,5
8,"{u'mana': 1, u'id': u'LOOT_542', u'name': u'Ki...",me,5
9,"{u'mana': 1, u'id': u'CS2_074', u'name': u'Dea...",me,5


In [18]:
sentances

[u"Open_the_Waygate Arcanologist Arcane_Intellect Ghastly_Conjurer The_Coin Mirror_Image Ice_Barrier Novice_Engineer Coldlight_Oracle Ice_Block Primordial_Glyph Arcanologist Coldlight_Oracle Novice_Engineer Doomsayer Simulacrum Frost_Nova Arcane_Intellect Ghastly_Conjurer Mirror_Image Blizzard Doomsayer Fireblast Sorcerer's_Apprentice Sorcerer's_Apprentice Sorcerer's_Apprentice Molten_Reflection Molten_Reflection Cabalist's_Tome Greater_Arcane_Missiles Frozen_Clone Time_Warp Archmage_Antonidas Primordial_Glyph Fireball Fireball",
 u"Fireblast Tar_Creeper Golakka_Crawler Fireblast Acolyte_of_Pain Fireblast Ice_Barrier Fireblast Water_Elemental Doomsayer Tol'vir_Stoneshaper Tol'vir_Stoneshaper Tar_Creeper Fireblast Servant_of_Kalimos Servant_of_Kalimos Pyroblast",
 u'The_Caverns_Below Fire_Fly Gadgetzan_Ferryman Fire_Fly Zola_the_Gorgon The_Coin Vanish Glacial_Shard Flame_Elemental Fire_Fly Youthful_Brewmaster Fire_Fly Shadowstep Glacial_Shard Fire_Fly Crystal_Core Flame_Elemental Stonet